### 국회의원 정보 스크래핑
* 이름, ID 추출
* ID를 사용해서 상세페이지로 요청 보내기
* DataFrame에 저장하고, DB에 저장
* 시각화

In [46]:
import requests
from bs4 import BeautifulSoup
import re

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do'
req_param = {
    #?currentPage=1&rowPerPage=500
    "currentPage":1,
    "rowPerPage":500
}

res = requests.get(url, params=req_param)
print(res.status_code)

if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    atag_list = soup.select("div.memberna_list dl dt a")
    
    member_list = []    
    for atag in atag_list:

        member_dict = {}
        
        member_dict['name'] = atag.text  #<a href="">둘리</a>
        
        href = atag['href']
        matched = re.search(r'(\d+)', href)
        if matched:
            member_dict['id'] = matched.group(0)
        
        #print(member_dict)
        member_list.append(member_dict)

print(len(member_list))
member_list[0:2]

200
297


[{'name': '강기윤', 'id': '9771230'}, {'name': '강대식', 'id': '9771142'}]

In [48]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

print('----- 시작')

#모든 국회의원 정보를 저장할 list 선언
member_detail_list = []

for idx,member in enumerate(member_list,1):
    print(f'---> 번호 {idx} <------')
    member_id = member['id']
    detail_url = f'https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={member_id}'
    
    res = requests.get(detail_url)
    if res.ok:
        soup = BeautifulSoup(res.text, 'html.parser')
        
        #1명의 국회의원 정보를 저장할 dict를 선언
        member_dict = dict()
        
        dt_list = []
        for dt in soup.select("div.info_mna dl.pro_detail dt"):
            dt_list.append(dt.text)
        
        dd_list = []
        for dd in soup.select("div.info_mna dl.pro_detail dd"):
            #양쪽의 공백 제거
            dd_text = dd.text.strip()
            
            #Pattern 객체생성
            reg_exr = re.compile(r'[\n\r\t]')
            #sub()는 Pattern과 일치하는 문자을 empty string으로 대체해라
            #replace()는 white space를 empty string으로 변경해라
            clean_text = reg_exr.sub('', dd_text).replace(" ","")
            dd_list.append(clean_text)
        
        member_dict = dict(zip(dt_list,dd_list))
        #print(member_dict)
        
        for div_tag in soup.select("div.info_mna div.profile"):
            #print(div_tag)
            member_dict["이름"] = div_tag.find('h4').text
            
            img_src = div_tag.select('img')[0]['src']
            img_url = urljoin(detail_url, img_src)
            member_dict["이미지"] = img_url
            
            birth_date = div_tag.select_one("li:nth-of-type(4)").text
            member_dict["생년월일"] = birth_date
            
        #print(member_dict)
        member_detail_list.append(member_dict)   

print('===> 갯수 ', len(member_detail_list))
member_detail_list[296:]
print('----- 끝')        

----- 시작
---> 번호 1 <------
---> 번호 2 <------
---> 번호 3 <------
---> 번호 4 <------
---> 번호 5 <------
---> 번호 6 <------
---> 번호 7 <------
---> 번호 8 <------
---> 번호 9 <------
---> 번호 10 <------
---> 번호 11 <------
---> 번호 12 <------
---> 번호 13 <------
---> 번호 14 <------
---> 번호 15 <------
---> 번호 16 <------
---> 번호 17 <------
---> 번호 18 <------
---> 번호 19 <------
---> 번호 20 <------
---> 번호 21 <------
---> 번호 22 <------
---> 번호 23 <------
---> 번호 24 <------
---> 번호 25 <------
---> 번호 26 <------
---> 번호 27 <------
---> 번호 28 <------
---> 번호 29 <------
---> 번호 30 <------
---> 번호 31 <------
---> 번호 32 <------
---> 번호 33 <------
---> 번호 34 <------
---> 번호 35 <------
---> 번호 36 <------
---> 번호 37 <------
---> 번호 38 <------
---> 번호 39 <------
---> 번호 40 <------
---> 번호 41 <------
---> 번호 42 <------
---> 번호 43 <------
---> 번호 44 <------
---> 번호 45 <------
---> 번호 46 <------
---> 번호 47 <------
---> 번호 48 <------
---> 번호 49 <------
---> 번호 50 <------
---> 번호 51 <------
---> 번호 52 <------
---> 번호 53 <

In [50]:
import json

with open('data/member.json','w',encoding='utf8') as file:
    json.dump(member_detail_list, file)

In [51]:
import pandas as pd

member_df = pd.read_json('data/member.json')
print(member_df.shape)
member_df.head(2)

(297, 16)


,정당,선거구,소속위원회,당선횟수,사무실 전화,사무실 호실,홈페이지,이메일,보좌관,비서관,비서,"취미, 특기",의원실 안내,이름,이미지,생년월일
0,국민의힘,경남창원시성산구,보건복지위원회,"재선(19대,21대)",02-784-1751,의원회관937호,http://blog.naver.com/ggotop,ggotop@naver.com,"김홍광,한영애","지상훈,최광림","김영록,안효상,이유진,홍지형,김지훈",,,강기윤,https://www.assembly.go.kr/photo/9771230.jpg,1960-06-04
1,국민의힘,대구동구을,국방위원회,초선(21대),02-784-5275,의원회관341호,,kds21341@naver.com,"박홍규,정운태","유진영,윤미라","박순권,김광연,윤선영,김현정,송민욱",,,강대식,https://www.assembly.go.kr/photo/9771142.jpg,1959-11-02
